# Lab 4: Urban Heat Island Detection using Thermal Satellite Imagery

In [50]:
import geemap
import ee
import pandas as pd

ee.Authenticate()
ee.Initialize(project='rsia-lab')

## Goal
In this exercise, you will:
1. Select and compare two thermal satellite images from Kraków (or another city that has recently undergone intensive, thoughtless concrete development) – one from **2013** and one from **2024**.
2. Each image must be acquired on a **hot summer day** (T > 27°C based on IMGW meteorological data).
3. Images must have **low cloud cover** (< 20%).
4. Visualize thermal data and analyze surface temperature differences (Urban Heat Island effect).

## Task 1: Area of Interest (AOI)
- Define an AOI over the Reduta Street area in Kraków.
- Use `ee.Geometry.Polygon` or `ee.Geometry.Rectangle`.

In [51]:
aoi = ee.Geometry.Rectangle([19.791, 50.051, 19.902, 50.102]) #Kraków

## Task 2: Download and Analyze IMGW Meteorological Data
- Visit: [IMGW Archive](https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/)
- Navigate to:
  - `dane_meteorologiczne/dobowe/synop/2013/`
  - `dane_meteorologiczne/dobowe/synop/2024/`
- Download `s_d_tmax.csv` for both years.
- Filter the rows for **station ID 12566 (e.g. Kraków-Balice)**.
- Identify days in **July or August** with **TMAX > 27°C**.

In [52]:
file_path_2013 = "Data-lab-4/s_d_566_2013.csv" 
file_path_2024 = "Data-lab-4/s_d_566_2024.csv" 

data_2013 = pd.read_csv(file_path_2013, encoding='ISO-8859-1') 
data_2024 = pd.read_csv(file_path_2024, encoding='ISO-8859-1') 

filtered_data_2013 = data_2013[data_2013['10.3'] > 27]
filtered_data_2024 = data_2024[data_2024['4.7'] > 27]

print("Filtered data for 2013:")
print(filtered_data_2013)
print("Filtered data for 2024:")
print(filtered_data_2024)

Filtered data for 2013:
     350190566  KRAKÓW-BALICE  2013  01  01.1  10.3  Unnamed: 6  -4.7  \
166  350190566  KRAKÓW-BALICE  2013   6    17  27.6         NaN  15.2   
167  350190566  KRAKÓW-BALICE  2013   6    18  32.4         NaN  16.2   
168  350190566  KRAKÓW-BALICE  2013   6    19  27.1         NaN  18.2   
169  350190566  KRAKÓW-BALICE  2013   6    20  31.1         NaN  16.9   
170  350190566  KRAKÓW-BALICE  2013   6    21  33.4         NaN  18.7   
171  350190566  KRAKÓW-BALICE  2013   6    22  27.2         NaN  18.0   
182  350190566  KRAKÓW-BALICE  2013   7     3  28.3         NaN  12.3   
183  350190566  KRAKÓW-BALICE  2013   7     4  28.3         NaN  14.2   
184  350190566  KRAKÓW-BALICE  2013   7     5  27.1         NaN  13.7   
189  350190566  KRAKÓW-BALICE  2013   7    10  27.7         NaN  12.9   
201  350190566  KRAKÓW-BALICE  2013   7    22  27.7         NaN  11.9   
205  350190566  KRAKÓW-BALICE  2013   7    26  28.9         NaN  13.9   
206  350190566  KRAKÓW-BALI

## Task 3: Select Landsat 8 Images Matching These Dates
- In Earth Engine, use `LANDSAT/LC08/C02/T1_L2` collection.
- Apply filters:
  - `.filterBounds(aoi)`
  - `.filterDate()` for the matching day
  - `.filterMetadata('CLOUD_COVER', 'less_than', 20)`
- Try to find **one image from 2013** and **one image from 2024** that match your hot days list.

In [53]:
image_2013 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2013-07-26', '2013-08-09') \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .first()

image_2024 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2024-06-18', '2024-07-09') \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .first()

metadata_2013 = image_2013.getInfo()
metadata_2024 = image_2024.getInfo()

print("Cloudy pixel percentage for 2013 image:", metadata_2013['properties']['CLOUD_COVER'])
print("Cloudy pixel percentage for 2024 image:", metadata_2024['properties']['CLOUD_COVER'])

Cloudy pixel percentage for 2013 image: 0.54
Cloudy pixel percentage for 2024 image: 24.28


## Task 4: Process Thermal Band (ST_B10)
- Convert Band 10 to Brightness Temperature (Kelvin):
  `TB = ST_B10 * 0.00341802 + 149.0`
- Create a visualization of each image using the same color scale.

In [54]:
kalvin_2013 = image_2013.select('ST_B10').multiply(0.03418802).add(149.0)
kalvin_2024 = image_2024.select('ST_B10').multiply(0.03418802).add(149.0)
kalvin_diff = kalvin_2024.subtract(kalvin_2013)

metadata_kalvin_2013 = kalvin_2013.getInfo()
metadata_kalvin_2024 = kalvin_2024.getInfo()
metadata_kalvin_diff = kalvin_diff.getInfo()

# print("Metadata for 2013 image:", metadata_kalvin_2013['bands'][0]['data_type'])
# print("Metadata for 2024 image:", metadata_kalvin_2024['bands'][0]['data_type'])
# print("Metadata for difference image:", metadata_kalvin_diff['bands'][0]['data_type'])

Map = geemap.Map()
Map.centerObject(aoi)
Map.addLayer(kalvin_2013, {
    'min': metadata_kalvin_2013['bands'][0]['data_type']['min'],
    'max': metadata_kalvin_2013['bands'][0]['data_type']['max'],
    'palette': ['blue', 'green', 'yellow', 'red']
}, 'Temperature 2013')

Map.addLayer(kalvin_2024, {
    'min': metadata_kalvin_2024['bands'][0]['data_type']['min'],
    'max': metadata_kalvin_2024['bands'][0]['data_type']['max'],
    'palette': ['blue', 'green', 'yellow', 'red']
}, 'Temperature 2024')

Map.addLayer(kalvin_diff, {
    'min': metadata_kalvin_diff['bands'][0]['data_type']['min'],
    'max': metadata_kalvin_diff['bands'][0]['data_type']['max'],
    'palette': ['blue', 'green', 'yellow']
}, 'Temperature Difference 2013-2024')

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Task 5: Compare and Interpret
- Compare the two maps.
- Optionally calculate difference: `TB_2024 - TB_2013`
- Discuss: did the surface temperature increase in the area?
- Is there evidence of an Urban Heat Island effect related to development?

Despite significant development, the surfer temperature was definitely higher in 2013, particularly in the area near Krakow. There were also more Urban Heat Islands compared to 2024. However, some Urban Heat Islands can still be observed in the centre. As shown on the temperature difference map, the general temperature was quite similar. It’s difficult to say if the Urban Heat Island is directly related to development; while there are some places where this is true, in general, there wasn’t a significant change over the 10-year period. In my opinion, more analysis and data are needed to draw definitive conclusions.